In [52]:
import requests
import pandas as pd
import json
import time
from tqdm import tqdm
from vt_api_key import api_key

In [86]:
train_data = pd.read_csv('data/processed/sample_train.csv')

C:\Users\ilanv\AppData\Local\Temp/ipykernel_31604/1024237520.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('data/processed/sample_train.csv')


In [92]:
train_malwares = pd.DataFrame(train_data[train_data['label'] == 1]['sha256'].copy())
train_malwares['status_code'] = 0
train_malwares['timestamp'] = None

In [117]:
# DO NOT RUN THIS CODE!
# train_malwares.to_csv('data/virustotal/train_file_status.csv', index=False)

In [118]:
# DO NOT RUN THIS CODE!
# with open('data/virustotal/train_file_data.json', 'w') as f:
    # json.dump({}, f)

In [13]:
# index of last file that wasn't checked
last_index = train_malwares[train_malwares['status_code'] == 0].index[0]
last_index

1001

In [27]:
file_hash = 'moti'
url = f"https://www.virustotal.com/api/v3/files/{file_hash}"

headers = {
    "accept": "application/json",
    "x-apikey": "5adf3682d43ee1cb47ffc91c8e04e520be5637f4df6a1fee1190d964b806f4fa"
}

response = requests.get(url, headers=headers)

print(response.text)

{"error":{"code":"NotFoundError","message":"Resource not found."}}



In [23]:
def get_remaning_daily_quota(api_key):
    url = f"https://www.virustotal.com/api/v3/users/{api_key}/overall_quotas"

    headers = {
        "accept": "application/json",
        "x-apikey": api_key
    }

    response = requests.get(url, headers=headers)
    user_daily_quota = response.json()['data']['api_requests_daily']['user']
    used = user_daily_quota['used']
    allowed = user_daily_quota['allowed']
    return allowed - used

In [14]:
url = f"https://www.virustotal.com/api/v3/users/{api_key}/overall_quotas"

headers = {
    "accept": "application/json",
    "x-apikey": api_key
}

response = requests.get(url, headers=headers)

In [56]:
get_remaning_daily_quota(api_key)

19936

In [113]:
def create_batches(start_index, df_size):
    # num_avilable_requests = get_remaning_daily_quota(api_key)
    num_avilable_requests = 20000
    num_requests = 0 
    batches = []
    while num_requests < num_avilable_requests and start_index < df_size:
        batch_samples = min([3, num_avilable_requests - num_requests, df_size - start_index])
        end_index = start_index + batch_samples 
        batches.append(range(start_index, end_index))
        num_requests += 3
        start_index = end_index
    return batches

In [81]:
batches = create_batches(train_malwares, 0)
batches[0]

range(0, 3)

In [82]:
data_json = {}
batches = create_batches(train_malwares, 0)
for b in range(len(batches)):
    for idx in tqdm(batches[b], desc = f"batch {b}"):
        
        sha256 = train_malwares.loc[idx, 'sha256']
        # get response
        url = f"https://www.virustotal.com/api/v3/files/{sha256}"
        headers = {
            "accept": "application/json",
            "x-apikey": api_key
        }
        response = requests.get(url, headers=headers)
        # update status and timestamp
        train_malwares.loc[idx, 'timestamp'] = pd.Timestamp.now()
        train_malwares.loc[idx, 'status_code'] = response.status_code
        # save response json
        data_json[sha256] = response.json()

    # save progress
    train_malwares.to_csv('data/virustotal/train_file_status.csv', index=False)
    with open('data/virustotal/train_file_data.json', 'w') as f:
            json.dump(data_json, f)

    if b != len(batches) - 1:
        # sleep for 1 minute
        time.sleep(3)



        

batch 3: 100%|██████████| 2/2 [00:01<00:00,  1.67it/s]


In [85]:
# open json file
with open('data/virustotal/train_file_data.json', 'r') as f:
    data_json_shmulik = json.load(f)
len(data_json_shmulik)

11

In [19]:
# get number of malicious and undected votes 
malicious_votes = response.json()['data']['attributes']['last_analysis_stats']['malicious']
undetected_votes = response.json()['data']['attributes']['last_analysis_stats']['undetected']
response.json()['data']['attributes']['last_analysis_stats']

{'harmless': 0,
 'type-unsupported': 4,
 'suspicious': 0,
 'confirmed-timeout': 0,
 'timeout': 0,
 'failure': 5,
 'malicious': 53,
 'undetected': 9}

In [21]:
json_dic = {}
json_dic[file_hash] = response.json()
# convert to json 
json_dic = json.dumps(json_dic) 

In [28]:
# get response code
response.status_code

404

In [114]:
len(train_malwares)
batches = create_batches(79950, len(train_malwares))